In [1]:
from datasets import Dataset
classes = ["banking","valuation","household","real estate","corporate","external","sovereign","technology", "climate", "energy", "health", "eu"]


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support,top_k_accuracy_score
import math
import pickle

In [3]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(classes))
finbert = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert')
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert', use_fast =True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [4]:
# object_methods = [method_name for method_name in dir(finbert)
#                   if callable(getattr(finbert, method_name))]
# print(object_methods)
finbert_weights = finbert.state_dict()
model_weights = model.state_dict()
del finbert_weights["bert.pooler.dense.weight"]
del finbert_weights["bert.pooler.dense.bias"]
del finbert_weights["classifier.weight"]
del finbert_weights["classifier.bias"]
finbert_weights["bert.pooler.dense.weight"] = model_weights["bert.pooler.dense.weight"]
finbert_weights["bert.pooler.dense.bias"] = model_weights["bert.pooler.dense.bias"]
finbert_weights["classifier.weight"] = model_weights["classifier.weight"]
finbert_weights["classifier.bias"] = model_weights["classifier.bias"]

model.load_state_dict(finbert_weights)


<All keys matched successfully>

In [5]:
# flatten to one list for all 3
with open('train_data.pickle', 'rb') as file:
    train = pickle.load(file)

with open('gpt.pickle', 'rb') as file:
    gpt = pickle.load(file)

with open('gpt_p2.pickle', 'rb') as file:
    gpt2 = pickle.load(file)
    
gpt = [item for sublist in gpt for item in sublist]
gpt2 = [item for sublist in gpt2 for item in sublist]

mixed = gpt + gpt2

In [6]:
print(len(mixed))

2458


In [7]:
sample = 10
# text = [item["text"] for i in range(sample) for item in mixed]
# label = [np.random.choice(len(classes), p=item["dist"]) for i in range(sample) for item in mixed]
# additional_text = []
# additional_label = []

# for idx in range(1,len(mixed)-1):
#     sent1, sent2 = mixed[idx-1:idx+1]
#     for i in range(3):
#         additional_text.append(sent1["text"]+' '+ sent2["text"])
#         dist = (np.array(sent1["dist"]) + np.array(sent2["dist"]))/2
#         additional_label.append(np.random.choice(len(classes), p=dist))

# for idx in range(2,len(mixed)-1):
#     sent1, sent2, sent3 = mixed[idx-2:idx+1]
#     for i in range(2):
#         additional_text.append(sent1["text"]+' '+ sent2["text"]+' '+sent3["text"])
#         dist = (np.array(sent1["dist"]) + np.array(sent2["dist"])+ np.array(sent3["dist"]))/3
#         additional_label.append(np.random.choice(len(classes), p=dist))

text_max = [item["text"] for i in range(sample) for item in train]
label_max = [np.argmax(item["dist"]) for i in range(sample) for item in train]

text_max_mixed = [item["text"] for i in range(sample-1) for item in mixed]
label_max_mixed = [np.argmax(item["dist"]) for i in range(sample-1) for item in mixed]

# comb_text = text + additional_text + text_max
# comb_label = label + additional_label + label_max
comb_text = text_max+text_max_mixed
comb_label = label_max+label_max_mixed

In [8]:

# with open('dataset_57.pickle', 'wb') as file:
#     pickle.dump((comb_text,comb_label), file)

In [9]:

# with open('dataset_57.pickle', 'rb') as file:
#     comb_text,comb_label = pickle.load(file)

In [10]:
assert len(comb_text) == len(comb_label)

In [11]:
import random
temp = list(zip(comb_text, comb_label))
random.shuffle(temp)
comb_text, comb_label = zip(*temp)

In [12]:
print(len(comb_label))

39468


In [13]:
train_text, test_text = comb_text[:math.ceil(len(comb_text)*0.9)], comb_text[:math.ceil(len(comb_text)*0.1)]
train_label, test_label = comb_label[:math.ceil(len(comb_label)*0.9)], comb_label[:math.ceil(len(comb_label)*0.10)]

In [14]:
train_dataset = Dataset.from_dict({"text":train_text, "label":train_label})
test_dataset = Dataset.from_dict({"text":test_text, "label":test_label})

In [15]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

100%|██████████| 1/1 [00:00<00:00,  4.04ba/s]


In [16]:
train_dataset["input_ids"]

tensor([[  101,  2023,  3663,  ...,     0,     0,     0],
        [  101,  1996,  3189,  ...,     0,     0,     0],
        [  101, 10940,  2005,  ...,     0,     0,     0],
        ...,
        [  101,  1996,  3006,  ...,     0,     0,     0],
        [  101,  2017,  2097,  ...,     0,     0,     0],
        [  101,  2107, 14437,  ...,     0,     0,     0]])

In [17]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    top3 = top_k_accuracy_score(labels, pred.predictions,k=3)
    top2 = top_k_accuracy_score(labels, pred.predictions,k=2)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'top3': top3,
         'top2': top2
    }

training_args = TrainingArguments(
    
    output_dir='./results',
    learning_rate=2e-5,
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    metric_for_best_model="accuracy",
    evaluation_strategy='epoch',
    save_strategy = "epoch",
    logging_dir='./logs',
    save_total_limit = 1, # Only last 5 models are saved. Older ones are deleted.
    load_best_model_at_end=True,
)
    # "test-glue",
    # evaluation_strategy = "epoch",
    # save_strategy = "epoch",
    # learning_rate=2e-5,
    # per_device_train_batch_size=batch_size,
    # per_device_eval_batch_size=batch_size,
    # num_train_epochs=5,
    # weight_decay=0.01,
    # load_best_model_at_end=True,
    # metric_for_best_model=metric_name,
    # push_to_hub=True,
    # push_to_hub_model_id=f"{model_name}-finetuned-{task}",
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [18]:
trainer.train()

  1%|          | 501/44420 [01:18<1:55:13,  6.35it/s]

{'loss': 2.3264, 'learning_rate': 2e-05, 'epoch': 0.23}


  2%|▏         | 1001/44420 [02:37<1:53:49,  6.36it/s]

{'loss': 2.0445, 'learning_rate': 1.9772313296903462e-05, 'epoch': 0.45}


  3%|▎         | 1501/44420 [03:56<1:52:13,  6.37it/s]

...


100%|██████████| 44420/44420 [1:57:29<00:00,  6.60it/s]

{'eval_loss': 1.0121009349822998, 'eval_accuracy': 0.5741069166455536, 'eval_f1': 0.5288096905676282, 'eval_precision': 0.5445052841552933, 'eval_recall': 0.5186075178894037, 'eval_top3': 0.8821890043070687, 'eval_top2': 0.7894603496326323, 'eval_runtime': 9.4118, 'eval_samples_per_second': 419.365, 'epoch': 20.0}


100%|██████████| 44420/44420 [1:57:32<00:00,  6.30it/s]

{'train_runtime': 7052.3899, 'train_samples_per_second': 6.299, 'epoch': 20.0}


TrainOutput(global_step=44420, training_loss=1.3842319218868717, metrics={'train_runtime': 7052.3899, 'train_samples_per_second': 6.299, 'epoch': 20.0})

In [19]:
trainer.evaluate()

100%|██████████| 62/62 [00:09<00:00,  6.72it/s]


{'eval_loss': 1.0243563652038574,
 'eval_accuracy': 0.5766404864454016,
 'eval_f1': 0.5316538589564456,
 'eval_precision': 0.546277783485254,
 'eval_recall': 0.5233829825328863,
 'eval_top3': 0.8826957182670383,
 'eval_top2': 0.7907271345325564,
 'eval_runtime': 9.3876,
 'eval_samples_per_second': 420.447,
 'epoch': 20.0}